In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree

rng = np.random.RandomState(0)

In [2]:
df = pd.read_csv('Laptop-Preise.csv', sep=';', decimal=',')
df = df[df.extern_Schnittstellen != 2300] # Ausreißer löschen
df = df.reset_index(drop=True) # Reset Index
df.drop(['Betriebssystem_OHNE', 'Betriebssystem_Mac', 'Marke_Dell'], axis=1) # Mac und Marke_Apple sind identische Merkmale
df_noPrice = df.drop('Preis', axis=1)
error_mse = pd.DataFrame(columns=['chance', 'LM', 'KNN_1', 'KNN_3', 'KNN_5'])
error_stdErr = pd.DataFrame(columns=['chance', 'LM', 'KNN_1', 'KNN_3', 'KNN_5'])

In [46]:
train_values, test_values, train_labels, test_labels = train_test_split(df_noPrice, df['Preis'], test_size=0.01)

def del_ran(df, chance):
    rand_array = np.random.rand(df.shape[0])
    delete_entries = rand_array < chance
    keep_entries = rand_array >= chance
    return {'keep' : df.iloc[keep_entries], 'delete' : df.iloc[delete_entries]}
# Umschreiben, dass es auch mit der aktuellen impute-Methode funktioniert

In [4]:
temp = del_ran(df_noPrice, 0.01)
# temp['keep']
# temp['delete']

In [ ]:
k = temp['keep']
d = temp['delete']
t = d.iloc[0].values.reshape(1, -1)

print(len(t))
print(k.shape)

tree = KDTree(temp['keep'], leaf_size=5)              
dist, ind = tree.query(t, k=3)


print(dist)
print(ind[0])
# [641 501 201]

pd.DataFrame(t)
pd.DataFrame(k.at(ind[0]))

# for t in d.iterrows():
#     dist, ind = tree.query(t, k=3)
#     print(ind, dist)


In [85]:
tree = KDTree(train_values.values, leaf_size=5)

def impute():
    imputed_values = []

    for index, entry in enumerate(test_values.values):
        # print(entry)
        dist, ind = tree.query([entry], k=5)
        ind = ind[0]

        current_impute_knn_1 = np.mean(train_labels.values[ind][0])
        current_impute_knn_3 = np.mean(train_labels.values[ind][:3])
        current_impute_knn_5 = np.mean(train_labels.values[ind])
        print(train_labels.values[ind])
        print(current_impute_knn_1, current_impute_knn_3, current_impute_knn_5)

        imputed_values.append(current_impute_knn_1)


        print(test_labels.values[index])
        print(np.mean(train_labels.values[ind]), '\n')
        # print(train_values.values[ind])

    errors = test_labels.values - imputed_values
    print(errors)

In [86]:
impute()

[2079. 2149. 2249. 2299. 1899.]
2079.0 2159.0 2135.0
2079.0
2135.0 

[919. 899. 899. 899. 999.]
919.0 905.6666666666666 923.0
969.0
923.0 

[2149. 1899.  899. 2079. 1729.]
2149.0 1649.0 1751.0
1699.0
1751.0 

[2199. 1379. 1379. 1379. 1379.]
2199.0 1652.3333333333333 1543.0
2129.0
1543.0 

[556. 399. 519. 429. 619.]
556.0 491.3333333333333 504.4
499.0
504.4 

[1399. 1179.  899. 1249.  899.]
1399.0 1159.0 1125.0
1249.0
1125.0 

[349. 739. 619. 749. 579.]
349.0 569.0 607.0
479.0
607.0 

[1729. 1499. 1099. 1929. 2079.]
1729.0 1442.3333333333333 1667.0
1349.0
1667.0 

[ 899. 1099.  999.  699.  999.]
899.0 999.0 939.0
1299.0
939.0 

[1079. 1629.  849.  979.  849.]
1079.0 1185.6666666666667 1077.0
899.0
1077.0 

[504. 549. 549. 599. 719.]
504.0 534.0 584.0
549.0
584.0 

[   0.   50. -450.  -70.  -57. -150.  130. -380.  400. -180.   45.]


In [6]:
tree = KDTree(df, leaf_size=5)              
dist, ind = tree.query(df[:1], k=5) # Bei den Queries wird der Index 1 als Startindex verwendet? --> Nein
print(ind)  # indices of 3 closest neighbors (Indizes bei 0 beginnend)
print(dist)

[[   0  281  227  115 1028]]
[[  0.          61.05644929 100.01499888 100.98955392 101.30429408]]
